In [ ]:
import os
import numpy as np
import pandas as pd

from itertools import product

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import copy

In [ ]:
matplotlib.rcParams.update({
    'font.size': 18,
    'legend.fontsize': 18,
    'legend.handlelength': 2
})

In [ ]:
folds = [1, 2, 3, 4, 5]

experiment = "comp"
model_types = []

name_mapper = {"CNN": "Arvalus", "GCN": "D-Arvalus"}
anomaly_mapper = {"0": "Normal", "2": "Anomaly 1", "5": "Anomaly 2", "6": "Anomaly 3", "8": "Anomaly 4"}

In [ ]:
path = "../../results/calculated_scores.csv"

df_list = []

if os.path.exists(path):
    temp_df = pd.read_csv(path)
    df_list.append(temp_df)
    model_types.append("CNN")
    model_types.append("GCN")

df = pd.concat(df_list, axis = 0, ignore_index = True)

fig = None

In [ ]:
best_df = pd.DataFrame()

for fold, model_type in product(folds, model_types):
    df_ = df.loc[(df['fold'] == fold) & (df['model_type'] == model_type)]
    max_idx = df_["bc_f1_avg"].idxmax(axis=0) 
        
    best_df = best_df.append(df.loc[max_idx], ignore_index=True)
print(len(best_df))

In [ ]:
palette = "muted"

scores_to_plot = ["Accuracy", "F1", "Precision", "Recall"]
aggr = {s: ["mean"] for s in scores_to_plot}
    
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,3))

renamed_scores = {"bc_acc": "Accuracy", "bc_f1_avg": "F1", "bc_prec_avg": "Precision", "bc_rec_avg": "Recall"}
cols_to_keep = [
    "bc_acc",
    "bc_f1_avg",
    "bc_prec_avg",
    "bc_rec_avg",
    "model_type",
    "fold"
]

df_plot = copy.deepcopy(best_df)
df_plot["model_type"] = df_plot["model_type"].apply(lambda val: name_mapper[val])
df_plot = df_plot.loc[:, cols_to_keep]
df_plot = df_plot.rename(columns=renamed_scores)
df_plot = df_plot.groupby(["model_type", "fold"]).agg(aggr)
df_plot.columns = df_plot.columns.droplevel(1)
df_plot = df_plot.stack()
df_plot = df_plot.reset_index(level=1).reset_index()
df_plot = df_plot.drop(["fold"], axis=1)

df_plot.columns = ["Model", "score_type", "score"]

print(df_plot.groupby(["Model", "score_type"]).mean())

sns.barplot(x="score_type", y="score", hue="Model", data=df_plot, ax=ax1, capsize=.15, ci="sd", palette=palette)
ax1.set_title("Anomaly Detection")
ax1.get_legend().remove()
ax1.set_xlabel("")
ax1.set_ylabel("")

renamed_scores = {"mc_acc": "Accuracy", "mc_f1_avg": "F1", "mc_prec_avg": "Precision", "mc_rec_avg": "Recall"}
cols_to_keep = [
    "mc_acc",
    "mc_f1_avg",
    "mc_prec_avg",
    "mc_rec_avg",
    "model_type",
    "fold"
]

df_plot = copy.deepcopy(best_df)
df_plot["model_type"] = df_plot["model_type"].apply(lambda val: name_mapper[val])
df_plot = df_plot.loc[:, cols_to_keep]
df_plot = df_plot.rename(columns=renamed_scores)
df_plot = df_plot.groupby(["model_type", "fold"]).agg(aggr)
df_plot.columns = df_plot.columns.droplevel(1)
df_plot = df_plot.stack()
df_plot = df_plot.reset_index(level=1).reset_index()
df_plot = df_plot.drop(["fold"], axis=1)
df_plot.columns = ["Model", "score_type", "score"]

print(df_plot.groupby(["Model", "score_type"]).mean())

sns.barplot(x="score_type", y="score", hue="Model", data=df_plot, ax=ax2, capsize=.15, ci="sd", palette=palette)
ax2.set_title("Anomaly Type Detection")
ax2.get_legend().remove()
ax2.set_xlabel("")
ax2.set_ylabel("")    


plt.xlabel("")
plt.ylabel("")
plt.ylim(0, 1.05)
plt.legend(loc='upper center', ncol=2, fancybox=True, shadow=True, bbox_to_anchor=(-0.115, 1.42))
fig.savefig("../../results/plots/overall_results_detection-classification.pdf", bbox_inches='tight')